In [ ]:
import pandas as pd
import xgboost as xgb

In [ ]:
TrTa=pd.read_csv('train_target.csv')

In [ ]:
TrTr=pd.read_csv('transactions_train.csv')

In [ ]:
qw=TrTr.drop('trans_date',axis=1).fillna(0).groupby(['client_id','small_group']).sum(level='amount_rur')

In [ ]:
am_sum=qw.reset_index().pivot(index='client_id',columns='small_group', values='amount_rur')
am_sum=am_sum.fillna(0)
am_sum.columns=['gr_sum'+str(i) for i in am_sum.columns]

In [ ]:
am_count=TrTr.groupby(['client_id','small_group'])['amount_rur'].count()
am_count=am_count.reset_index().pivot(index='client_id',columns='small_group', values='amount_rur')
am_count=am_count.fillna(0)
am_count.columns=['gr_count'+str(i) for i in am_count.columns]

In [ ]:
agg_features=TrTr.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()

In [ ]:
train=pd.merge(TrTa,agg_features, on='client_id')
train=pd.merge(train,am_sum.reset_index(), on ='client_id')

In [ ]:
test_id=pd.read_csv('test.csv')
TrTe=pd.read_csv('transactions_test.csv')

In [ ]:
test_features=TrTe.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()

In [ ]:
test_sum=TrTe.drop('trans_date',axis=1)
test_sum=test_sum.groupby(['client_id','small_group']).sum(level='amount_rur')
test_sum=test_sum.reset_index().pivot(index='client_id',columns='small_group', values='amount_rur')
test_sum=test_sum.fillna(0)
test_sum.columns=['gr_sum'+str(i) for i in test_sum.columns]

In [ ]:
test_count=TrTe.groupby(['client_id','small_group'])['amount_rur'].count()
test_count=test_count.reset_index().pivot(index='client_id',columns='small_group', values='amount_rur')
test_count=test_count.fillna(0)
test_count.columns=['gr_count'+str(i) for i in test_count.columns]

In [ ]:
test=pd.merge(test_id,test_features, on='client_id')
test=pd.merge(test,test_sum.reset_index(), on ='client_id')

In [ ]:
common_features=list(set(train.columns).intersection(set(test.columns)))

In [ ]:
y_train=train['bins']
X_train=train[common_features]
X_test=test[common_features]

In [ ]:
model=xgb.XGBClassifier(silent=False, nthread=10, max_depth=8, n_estimators=3400, learning_rate=0.006, seed=425)
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.006, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=3400, n_jobs=1,
              nthread=10, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=425,
              silent=False, subsample=1, verbosity=1)

In [ ]:
pred=model.predict(X_test)

In [ ]:
submission = pd.DataFrame({'bins': pred}, index=test.client_id)
submission.head()

,bins
client_id,
28571,0
27046,2
13240,0
19974,2
10505,1


In [ ]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('submissions'):
    os.makedirs('submissions')

print(submission_path)
submission.to_csv(submission_path, index=True)

submissions/1578402816.csv
